In [2]:
from mxnet import nd
from mxnet.contrib import text

# 检测提供的预训练词嵌入
text.embedding.get_pretrained_file_names().keys()

dict_keys(['glove', 'fasttext'])

In [6]:
# 查看所提供的模型
print(text.embedding.get_pretrained_file_names('glove'))

['glove.42B.300d.txt', 'glove.6B.50d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.840B.300d.txt', 'glove.twitter.27B.25d.txt', 'glove.twitter.27B.50d.txt', 'glove.twitter.27B.100d.txt', 'glove.twitter.27B.200d.txt']


In [8]:
glove_6b50d = text.embedding.create(
    'glove', pretrained_file_name='glove.6B.50d.txt')
zh = text.embedding.create('fasttext', pretrained_file_name='wiki.zh.vec')

d:\miniconda3\envs\gluon\lib\site-packages\mxnet\contrib\text\embedding.py:279: UserWarning: At line 1 of the pre-trained text embedding file: token 332647 with 1-dimensional vector [300.0] is likely a header and is skipped.
  'skipped.' % (line_num, token, elems))


In [4]:
len(glove_6b50d)

400001

In [10]:
zh.token_to_idx['你好'],zh.idx_to_token[17442]

(17442, '你好')

In [12]:
# 使用词典
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

In [13]:
# 返回k个近邻的索引和相似度
def knn(W, x, k):
    # 添加的1e-9是为了数值稳定性
    cos = nd.dot(W, x.reshape((-1,))) / (
        (nd.sum(W * W, axis=1) + 1e-9).sqrt() * nd.sum(x * x).sqrt())
    topk = nd.topk(cos, k=k, ret_typ='indices').asnumpy().astype('int32')
    return topk, [cos[i].asscalar() for i in topk]

In [14]:
# 获取相似的k个词
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec,
                    embed.get_vecs_by_tokens([query_token]), k+1)
    for i, c in zip(topk[1:], cos[1:]):  # 除去输入词
        print('cosine sim=%.3f: %s' % (c, (embed.idx_to_token[i])))

In [15]:
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [23]:
get_similar_tokens('儿子', 3, zh)

cosine sim=0.982: 女儿
cosine sim=0.976: 母亲
cosine sim=0.975: 孙子


In [9]:
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [24]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed.get_vecs_by_tokens([token_a, token_b, token_c]) # (3,50)
    # 关键
    x = vecs[1] - vecs[0] + vecs[2]
    # 
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[topk[0]]

In [16]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [17]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

In [18]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

In [19]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'

In [27]:
get_analogy('父亲', '母亲', '爷爷', zh)

'爷爷'